<h1>KidRSA Cryptography</h1>
<h3>Overview:</h3>
<ol>
    <li>Key Generation</li>
    <li>Encoding/Decoding Function</li>
    <li>Applying KidRSA</li>
    <li>Euclid's Algorithm</li>
    <li>Hacking KidRSA</li>
</ol>


<h3>Key Generation:</h3>

In [87]:
# Declaring some random initial variables
a  = 4
b = 81
a_prime = 123
b_prime = 19963

def kidrsa_keygen(a:int, b:int, a_prime:int, b_prime:int):
    """This is the key generation function for the KidRSA algorithm. It takes four integers as inputs
    and returns the public (e,n) and private (d,n) keys."""
    M = (a * b) - 1
    e = a_prime * M + a
    d = b_prime * M + b
    n = (e * d - 1) / M
    return (e,n), (d,n)

public_key, private_key = kidrsa_keygen(a, b, a_prime, b_prime)
print(f"Public Key: {public_key}")
print(f"Private Key: {private_key}")

Public Key: (39733, 793199843.0)
Private Key: (6448130, 793199843.0)


<h3>Encoding/Decoding Function:</h3>

In [88]:
def encode(input_string:str):
    """Encodes and returns the ordinal (numerical) value of the inputted string as a list"""
    out = []
    for char in input_string:
        out.append(ord(char))
    return out

In [89]:
def decode(input_list:list):
    """Decodes the list of ordinals passed in and returns a string of characters"""
    out = ""
    for num in input_list:
        out += chr(int(num))
    return out

In [90]:
encoding = encode("Hello World")
decoding = decode(encoding)
print(f"Encoded: {encoding}")
print(f"Decoded: {decoding}")

Encoded: [72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]
Decoded: Hello World


<h3>Appying KidRSA:</h3>

In [91]:
def kidrsa_single(key:tuple, ordinal:int):
    """A helper function that pplies the KidRSA algorithm to a single integer using the 
    key (can be either public or private key) and ordinal passed in. Returns the modulo 
    of the multiplied values."""
    x,n = key
    return (x * ordinal) % n

In [92]:
def kidrsa(key:tuple, message:str|list):
    """A full-fledged KidRSA algorithm. Due to its symmetric nature this function works for both encryption and decryption.
    Depending on the type of inputted message, the algorithm can decrypt a list of ordinals using a private key or encrypt a message
    using a public key. The returned output is either a list of ordinals or a string of characters."""
    out_list = []
    if type(message) == str:
        msg_list = encode(message)
    elif type(message) == list:
        msg_list = message[:]
    for x in msg_list:
        out_list.append(kidrsa_single(key, x))
    if type(message) == list:
        return decode(out_list)
    else:
        return out_list

In [93]:
msg = [2900509, 4609028, 1549587, 4569295, 1271456, 3854101, 1271456,
3973300, 3854101, 4370630, 4092499, 4013033, 4529562, 4410363,
4648761, 4569295, 1271456, 3893834, 4648761, 4569295, 4171965,
4370630, 4013033, 4569295, 4569295, 1748252, 1271456, 2781310,
4529562, 4410363, 3973300, 4410363, 1748252, 1271456, 4092499,
4410363, 4171965, 4370630, 4092499, 1271456, 4410363, 4648761,
4609028, 1271456, 4807693, 4410363, 4648761, 4529562, 1271456,
3973300, 4410363, 4410363, 4529562, 1827718]

decryption = kidrsa(private_key, msg)
print(f"Decrypted Message: {decryption}")

Decrypted Message: It's a dangerous business, Frodo, going out your door.


<h3>Euclid's Algorithm (GCD):</h3>

In [94]:
def gcd(n:int, e:int):
    """An implementation of Euclid's algorithm. This function iteratively uses the divisor 
    and remainder to find the greatest common divisor between the two integers passed in."""
    a,b = n, e
    q,r = divmod(a,b)
    while r != 0:
        a,b = b,r
        q,r = divmod(a,b)
    return b

print(gcd(24,12))

12


In [95]:
def enhanced_gcd(n:int, e:int):
    """An enhanced version of the gcd function. Given that gcd can be represented as a linear combination (i.e. s*a + t*b = gcd(a,b) 
    for some integers s and t), this function keeps track of all the t's computed. The function then returns the greatest common divisor
    along with the last t value (i.e. 'd', the other component used for the private key)."""
    t = [0, 1]
    a,b = n,e
    q,r = divmod(a,b)
    while r != 0:
        t.append(t[-2] - q * t[-1])
        a,b = b,r
        q,r = divmod(a,b)
    return (b, t[-1])

e,n = public_key
gcd,d = (enhanced_gcd(n, e))
print(f"Greatest Commone Divisor: {gcd}")
print(f"Computed d: {d}")

Greatest Commone Divisor: 1.0
Computed d: 6448130.0


<h3>Hacking KidRSA:</h3>

In [96]:
# Generating a pair of public and private keys using random values
public_key, private_key = kidrsa_keygen(10, 15, 201, 23)
print(f"Public Key: {public_key}")
print(f"Private Key: {private_key}")

# Creating a message to be encrypted
message = "This is a top seceret message than cannot be intercepted!"
encrypted_msg = kidrsa(public_key, message)
print(f"Encrypted message: {encrypted_msg}")

# Using the enhance gcd function to discover the gcd between "n" and "e" and the private key value "d"
e,n = public_key
gcd, d = enhanced_gcd(n, e)
print(f"Greatest Common Divisor between {n} (n) and {e} (e) is: {gcd}")
print(f"The computed value for d is: {d}")

# Using the computed "d" value to decrypt the message inputted
decrypted_msg = kidrsa((d,n), encrypted_msg)
print(f"Decrypted message using computed 'd': {decrypted_msg}")

Public Key: (29959, 692073.0)
Private Key: (3442, 692073.0)
Encrypted message: [440337.0, 347444.0, 377403.0, 676993.0, 266615.0, 377403.0, 676993.0, 266615.0, 137731.0, 266615.0, 14879.0, 557157.0, 587116.0, 266615.0, 676993.0, 257567.0, 197649.0, 257567.0, 647034.0, 257567.0, 14879.0, 266615.0, 497239.0, 257567.0, 676993.0, 676993.0, 137731.0, 317485.0, 257567.0, 266615.0, 14879.0, 347444.0, 137731.0, 527198.0, 266615.0, 197649.0, 137731.0, 527198.0, 527198.0, 557157.0, 14879.0, 266615.0, 167690.0, 257567.0, 266615.0, 377403.0, 527198.0, 14879.0, 257567.0, 647034.0, 197649.0, 257567.0, 587116.0, 14879.0, 257567.0, 227608.0, 296574.0]
Greatest Common Divisor between 692073.0 (n) and 29959 (e) is: 1.0
The computed value for d is: 3442.0
Decrypted message using computed 'd': This is a top seceret message than cannot be intercepted!
